In [1]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
%matplotlib inline
le = LabelEncoder()

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
sample = pd.read_csv('SF_crime/sampleSubmission.csv')
test = pd.read_csv('SF_crime/test.csv', index_col='Id')
test = test.rename(columns={'X': 'Longitude', "Y": "Latitude"})
test.Dates = pd.to_datetime(test.Dates)
test_keep = test
crime_in_sf = pd.read_csv('SF_crime/train.csv')
crime_in_sf.Dates = pd.to_datetime(crime_in_sf.Dates)
crime_in_sf = crime_in_sf.rename(columns={'X': 'Longitude', "Y": "Latitude",})
crime_in_sf = crime_in_sf.drop(['Resolution', 'Descript'], axis=1)
crime_train, crime_test = train_test_split(crime_in_sf, test_size=.4)

In [3]:
for column in test.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(test[column])
        test[column] = le.transform(test[column])

for column in crime_in_sf.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(crime_in_sf[column])
        crime_train[column] = le.transform(crime_train[column])

for column in crime_in_sf.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(crime_in_sf[column])        
        crime_test[column] = le.transform(crime_test[column])

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
categories = crime_train.Category
crime_train = crime_train.drop('Category', axis=1)

categories2 = crime_test.Category
crime_test = crime_test.drop('Category', axis=1)

In [5]:
dtrain = xgb.DMatrix(crime_train.as_matrix(),
                     label=categories)
dtest = xgb.DMatrix(crime_test.as_matrix(),
                    label=categories2)

In [6]:
param = {'bst:max_depth':2, 'bst:eta':1, 'silent':1, 'objective':'multi:softprob', 'num_class':39}
param['nthread'] = 4
param['eval_metric'] = ['logloss']

In [7]:
evallist  = [(dtest,'eval'), (dtrain,'train')]

In [8]:
num_round = 25
bst = xgb.train(param, 
                dtrain, 
                num_round)

In [9]:
predictions = bst.predict(xgb.DMatrix(test.as_matrix()), output_margin=False)

In [10]:
predictions = pd.DataFrame(predictions)

In [11]:
le.fit(crime_in_sf.Category)
predictions.columns = le.inverse_transform(predictions.columns)

In [12]:
predictions['Id'] = predictions.index

In [13]:
def order(frame,var):
    varlist =[w for w in frame.columns if w not in var]
    frame = frame[var+varlist]
    return frame
predictions = order(predictions,['Id'])

In [ ]:
predictions

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.002615,0.139055,0.000156,0.000964,0.047462,0.002284,0.002780,0.031120,0.002859,...,0.000211,0.005159,0.000591,0.047435,1.463817e-05,0.008415,7.302221e-02,0.073508,4.234113e-02,0.022002
1,1,0.003826,0.135747,0.000178,0.000941,0.046333,0.002230,0.003121,0.058095,0.003196,...,0.000240,0.005036,0.000577,0.046306,5.487819e-05,0.008215,7.128499e-02,0.071759,4.133382e-02,0.021479
2,2,0.001970,0.105364,0.000305,0.000463,0.148079,0.003199,0.002009,0.016087,0.004773,...,0.000070,0.008827,0.000863,0.039129,1.493257e-05,0.010338,4.705526e-02,0.028051,4.174515e-02,0.008602
3,3,0.002715,0.106884,0.000218,0.000655,0.047026,0.001715,0.002539,0.025574,0.003162,...,0.000373,0.005257,0.000404,0.044349,9.905074e-06,0.004290,7.234362e-02,0.061784,1.936044e-02,0.011243
4,4,0.002715,0.106884,0.000218,0.000655,0.047026,0.001715,0.002539,0.025574,0.003162,...,0.000373,0.005257,0.000404,0.044349,9.905074e-06,0.004290,7.234362e-02,0.061784,1.936044e-02,0.011243
5,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001446,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
6,6,0.001419,0.090978,0.000126,0.000790,0.059681,0.001892,0.001857,0.028214,0.003047,...,0.000134,0.005578,0.000676,0.048926,4.414726e-06,0.004733,5.922588e-02,0.068160,2.135851e-02,0.006339
7,7,0.001610,0.103213,0.000174,0.000896,0.054720,0.002366,0.003179,0.032008,0.003456,...,0.000222,0.005426,0.000495,0.055506,3.665808e-06,0.005369,6.719085e-02,0.077326,3.563200e-02,0.009292
8,8,0.002515,0.079628,0.000317,0.001074,0.042045,0.007774,0.004086,0.055468,0.009913,...,0.000179,0.008662,0.000549,0.041717,2.401552e-05,0.008389,5.719588e-02,0.045337,5.058371e-02,0.017962
9,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000


In [ ]:
predictions.to_csv('predictions_XGB2.csv', index=False)

In [ ]:
xgb.plot_importance(bst)